In [ ]:
import os

path = r'C:\Users\Kacper\Desktop\RO\Lab09'

os.chdir(path)

# Optical Flow Lucas-Kanade 

In [ ]:
import numpy as np
import cv2

In [ ]:
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

maxLevel - patrz https://en.wikipedia.org/wiki/Pyramid_(image_processing) 
10 - iteracje
0.03 - epsilon

In [ ]:
lk_params = dict(winSize=(200,300),maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TermCriteria_COUNT, 10,0.03))

In [ ]:
cap = cv2.VideoCapture('highway2.mp4')
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

#Point to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)
mask = np.zeros_like(prev_frame)
while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        mask = cv2.line(mask, (int(x_new),int(y_new)),(int(x_prev),int(y_prev)),(0,255,0),3)
        frame = cv2.circle(frame, (int(x_new),int(y_new)),8,(0,0,255),-1)
    img = cv2.add(frame, mask)
    cv2.imshow('tracking',img)
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
        
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()


# Dense Optical Flow in OpenCV
algorytm Gunnar Farneback

calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags) -> flow


In [ ]:
import cv2 
import numpy as np

cap = cv2.VideoCapture('highway2.mp4')
ret, frame1 = cap.read()


prevImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prevImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # Convert back to BGR to show with imshow from cv
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    # Set the Previous image as the next iamge for the loop
    prevImg = nextImg

    
cap.release()
cv2.destroyAllWindows()

# Tracking in OpenCV

In [ ]:
tracker = cv2.TrackerMIL_create()
#tracker = cv2.TrackerKCF_create()

cap = cv2.VideoCapture('highway.mp4')

ret, frame3 = cap.read()

roi = cv2.selectROI(frame3, False)

ret = tracker.init(frame3, roi)

while True:
    ret, frame3 = cap.read()

    success, roi = tracker.update(frame3)
    
    (x,y,w,h) = tuple(map(int,roi))
    
    if success:
        # Tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)
        cv2.rectangle(frame3, p1, p2, (0,255,0), 3)
    else :
        # Tracking failure
        cv2.putText(frame3, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    cv2.imshow("tracking", frame3)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('highway2.mp4')
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

#Point to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)
mask = np.zeros_like(prev_frame)
while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        mask = cv2.line(mask, (int(x_new),int(y_new)),(int(x_prev),int(y_prev)),(0,255,0),3)
        frame = cv2.circle(frame, (int(x_new),int(y_new)),8,(0,0,255),-1)
    img = cv2.add(frame, mask)
    cv2.imshow('tracking',img)
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
        
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

In [ ]:
tracker = cv2.TrackerMIL_create()
#tracker = cv2.TrackerKCF_create()

cap = cv2.VideoCapture('highway2.mp4')

ret, frame3 = cap.read()

trackon = false

while True:
    ret, frame3 = cap.read()

    if(trackon):
        success, roi = tracker.update(frame3)

        (x,y,w,h) = tuple(map(int,roi))

        if success:
            # Tracking success
            p1 = (x, y)
            p2 = (x+w, y+h)
            cv2.rectangle(frame3, p1, p2, (0,255,0), 3)
        else :
            # Tracking failure
            cv2.putText(frame3, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    cv2.imshow("tracking", frame3)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 :
        break
        
    if k == 32 :
        roi = cv2.selectROI(frame3, False)
        ret = tracker.init(frame3, roi)
        
cap.release()
cv2.destroyAllWindows()